In [1]:
# 基于动态规划的维特比算法

# 例子： 经常有意见分歧
# 词典： [经常，经，有，有意见，意见，分歧，见，意，见分歧，分] 其他
# -logx [2.3, 3, 2.3, 2.3,  1.6,1.6,  3, 3,   3,  2.3] 20

# 第一步。首先创建有向权重图，采用矩阵的方式
class Graph:
    def __init__(self,node_num,default_flag=None):
        '''
        node_num:节点的个数，== len(sentence) + 1
        default_flag:矩阵默认填充的值
        '''
        self.graph_matrix = [[default_flag for i in range(node_num)] for i in range(node_num)]
        print('init matrix')
        self.print_matrix()
    
    def add_vertex(self,sentence,log_dict,max_window_len = 3,default_log=20.0):
        '''
        sentence:要添加边的句子
        log_dict:关于单词和-logp的字典
        max_window_len:最长的窗口长度
        '''
        
        seq_len = len(sentence)
        
        # 首先将句子切割
        for i in range(seq_len):
            char = sentence[i]
            weight = default_log
            if char in log_dict:
                weight = log_dict[char]
            self.graph_matrix[i][i+1] = weight
        print('struct single node')
        self.print_matrix()
            
        
        for win_len in range(2,max_window_len + 1):
            for start_index in range(0,seq_len - win_len + 1):
                char = sentence[start_index:start_index + win_len]
                if char in log_dict:
                    self.graph_matrix[start_index][start_index+ win_len] = log_dict[char]
        
        print('final matrix')
        self.print_matrix()
        
    def get_graph(self):
        return self.graph_matrix
    
    def print_matrix(self):
        for se_list in self.graph_matrix:
            print(se_list)

In [2]:
def init_graph(sentence,log_dict,default_log=20.0,default_flag=None):
    # 1.首先应该建立有向权重图
    seq_len = len(sentence)
    node_num = seq_len + 1
    graph = Graph(node_num,default_flag)
    # 设置最大窗口长度
    max_window_len = 3
    graph.add_vertex(sentence,log_dict,3,default_log)
    graph_matrix = graph.get_graph()
    return graph_matrix

In [16]:
def get_viterbi_from_graph(graph_matrix,node_num,default_flag):
    # 2. 动态规划解决viterbi问题
    # 存储动态规划的值
    dp = [float('Inf') for i in range(node_num)]
    dp[0] = 0.0
    # 存储从哪步跳转过来的节点
    route = [-1 for i in range(node_num)]
    # 动态规划解决
    for i in range(1,node_num):
        # 存储最优值的相关信息
        min_weight = float('inf')
        best_index = None
        # 确定到达i节点的边
        for j in range(i):
            if graph_matrix[j][i] != default_flag:
                if graph_matrix[j][i] + dp[j] < min_weight:
                    min_weight = graph_matrix[j][i] + dp[j]
                    best_index = j
        # 浮点数不精确，保留一位
        dp[i] = round(min_weight,1)
        print('%d ---> %s'%(i,str(dp[i])))
        route[i] = best_index
    return dp,route

In [4]:
def get_cut_result(sentence,dp,route):
    result_log = dp[-1]
    word_list = []
    seq_len = len(sentence)
    node_num = seq_len + 1
    
    start_index = node_num - 1
    while start_index > 0:
        prev_index = route[start_index]
        word_list.insert(0,sentence[prev_index:start_index])
        start_index = prev_index
        
    print('word_list:',str(word_list))
    print('result_log:',str(result_log))

In [8]:
def process(sentence,log_dict,defaut_log=20.0,default_flag=None):
    # 1.构建有向权重图
    graph_matrix = init_graph(sentence,log_dict,defaut_log,default_flag)
    # 2.viterbi算法构建
    dp,route = get_viterbi_from_graph(graph_matrix,len(sentence) + 1,default_flag)
    print('dp:',str(dp))
    print('route:',str(route))
    
    # 3.获取切词后的句子列表和log值
    get_cut_result(sentence,dp,route)

In [17]:
# 例子： 经常有意见分歧
# 词典： [经常，经，有，有意见，意见，分歧，见，意，见分歧，分] 其他
# -logx [2.3, 3, 2.3, 2.3,  1.6,1.6,  3, 3,   3,  2.3] 20

sentence = '经常有意见分歧'
log_dict = {
    '经常':2.3,
    '经':3.,
    '有':2.3,
    '有意见':2.3,
    '意见':1.6,
    '分歧':1.6,
    '见':3.,
    '意':3.,
    '见分歧':3.,
    '分':2.3
}

# sentence = '有意见分歧'
# log_dict = {
#     '有':2.3,
#     '有意见':2.3,
#     '意见':1.6,
#     '分歧':1.6,
#     '见':3.,
#     '意':3.,
#     '见分歧':3.,
#     '分':2.3
# }

process(sentence,log_dict,defaut_log=20.0,default_flag=None)

init matrix
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]
struct single node
[None, 3.0, None, None, None, None, None, None]
[None, None, 20.0, None, None, None, None, None]
[None, None, None, 2.3, None, None, None, None]
[None, None, None, None, 3.0, None, None, None]
[None, None, None, None, None, 3.0, None, None]
[None, None, None, None, None, None, 2.3, None]
[None, None, None, None, None, None, None, 20.0]
[None, None, None, None, None, None, None, None]
final matrix
[None, 3.0, 2.3, None, None, None, None, None]
[None, None, 20.0, None, None, None, None, None]
[None, None, None, 2.3, None, 2.3, None, None]
[None, None, None, None, 3.0, 1.6,

In [15]:
# float 无法精确表示
2.3 + 4.6

6.8999999999999995